In [1]:
pip install gql[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [2]:
import pandas as pd
import plotly.express as px
import json

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Polygon

## Balancer

In [12]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-polygon-v2-beta',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000
query1 = gql('''
query {
  poolContracts(
    first: 1000
    orderBy: pool__createTime
    orderDirection: asc
  ) {
    id
    pool {
      swapFee
      poolType
      tokens {
        address
        symbol
        decimals
      }
    }
  }
}
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['poolContracts'],max_level=1)
df12 = df11['pool.tokens'].map(lambda x:[i['address'] for i in x]).apply(pd.Series)
num_columns12 = len(df12.columns)
df12.columns = ['token{}'.format(i) for i in range(1, num_columns12+1)]
df13 = df11['pool.swapFee']
df1 = pd.concat([df11[['id','pool.poolType']], df13, df12], axis=1)
df1 = df1.rename(columns = {'id':'pool_address','pool.swapFee':'fee'})

#1000-2000
query2 = gql('''
query {
  poolContracts(
    first: 1000
    orderBy: pool__createTime
    orderDirection: asc
    skip: 1000
  ) {
    id
    pool {
      swapFee
      poolType
      tokens {
        address
        symbol
        decimals
      }
    }
  }
}
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['poolContracts'],max_level=1)
df22 = df11['pool.tokens'].map(lambda x:[i['address'] for i in x]).apply(pd.Series)
num_columns22 = len(df22.columns)
df22.columns = ['token{}'.format(i) for i in range(1, num_columns22+1)]
df23 = df21['pool.swapFee']
df2 = pd.concat([df21[['id','pool.poolType']], df23, df22], axis=1)
df2 = df2.rename(columns = {'id':'pool_address','pool.swapFee':'fee'})

#2000-3000
query3 = gql('''
query {
  poolContracts(
    first: 1000
    orderBy: pool__createTime
    orderDirection: asc
    skip: 2000
  ) {
    id
    pool {
      swapFee
      poolType
      tokens {
        address
        symbol
        decimals
      }
    }
  }
}
''')

response3 = client.execute(query3)

df31 = pd.json_normalize(response3['poolContracts'],max_level=1)
df32 = df31['pool.tokens'].map(lambda x:[i['address'] for i in x]).apply(pd.Series)
num_columns32 = len(df32.columns)
df32.columns = ['token{}'.format(i) for i in range(1, num_columns32+1)]
df33 = df31['pool.swapFee']
df3 = pd.concat([df31[['id','pool.poolType']], df33, df32], axis=1)
df3 = df3.rename(columns = {'id':'pool_address','pool.swapFee':'fee'})

#3000-4000
query4 = gql('''
query {
  poolContracts(
    first: 1000
    orderBy: pool__createTime
    orderDirection: asc
    skip: 3000
  ) {
    id
    pool {
      swapFee
      poolType
      tokens {
        address
        symbol
        decimals
      }
    }
  }
}
''')

response4 = client.execute(query4)

df41 = pd.json_normalize(response4['poolContracts'],max_level=1)
df42 = df41['pool.tokens'].map(lambda x:[i['address'] for i in x]).apply(pd.Series)
num_columns42 = len(df42.columns)
df42.columns = ['token{}'.format(i) for i in range(1, num_columns42+1)]
df43 = df41['pool.swapFee']
df4 = pd.concat([df41[['id','pool.poolType']], df43, df42], axis=1)
df4 = df4.rename(columns = {'id':'pool_address','pool.swapFee':'fee'})

#Combination
balancer_pools = pd.concat([df1,df2,df3,df4], ignore_index=True)
Name = pd.Series(["balancer" for x in range(len(balancer_pools.index))])
balancer_pools.insert(loc=0, column='Name', value=Name)
balancer_pools['fee'] = balancer_pools['fee'].apply(float)
balancer_pools['fee'] = balancer_pools['fee'].mul(100)
x = ['ERC4626Linear','AaveLinear','LiquidityBootstrapping','YearnLinear']
balancer_pools = balancer_pools[~balancer_pools['pool.poolType'].isin(x)]
balancer_pools = balancer_pools.drop('pool.poolType',axis=1).reset_index(drop=True)
balancer_pools

,Name,pool_address,fee,token1,token2,token3,token4,token5,token6,token7,token8,token9,token10,token11,token12,token13,token14
0,balancer,0x61d5dc44849c9c87b0856a2a311536205c96c7fd,0.20,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,balancer,0x647c1fd457b95b75d0972ff08fe01d7d7bda05df,1.00,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x85955046df4668e1dd369d2de9f3aeb98dd2a369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,balancer,0x0297e37f1873d2dab4487aa67cd56b58e2f27875,0.25,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,balancer,0xc6a5032dc4bf638e15b4a66bc718ba7ba474ff73,0.25,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,balancer,0x3a19030ed746bd1c3f2b0f996ff9479af04c5f0a,1.00,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x50b728d8d964fd00c2d0aad81718b71311fef68a,0x53e0bca35ec356bd5dddfebbd1fc0fd03fabad39,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3,0xd6df932a45c0f255f85145f286ea0b292b21c90b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2673,balancer,0xf7fd196c4606fb78178a85d1059f8224ddbccd2d,0.30,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0xa3fa99a148fa48d14ed51d610c367c61876997f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2674,balancer,0xacb0bd6f7206174c64cef5e05c4d7871ea888d33,1.00,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0xa486c6bc102f409180ccb8a94ba045d39f8fc7cb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2675,balancer,0x93a97f352c8be88d1468b8f7e0c24ce535be9eae,0.25,0x081ec4c0e30159c8259bad8f4887f83010a681dc,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2676,balancer,0xec7b8286e906fd74f14d0a0bdf0d362e0d1a3f9c,0.35,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0xe6ba37f375d11411565cb44c8a75e584483fbe1b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dfyn

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/dfyn-polygon',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, skip: 0, orderBy: reserve1, orderDirection: desc)
    {
      id
      reserve1
      token0 {
        id
        symbol
        decimals
      }
      token1 {
        id
        symbol
        decimals
      }
   }
}
''')

response1 = client.execute(query1)
dfyn_pool1 = pd.json_normalize(response1['pairs'],max_level=1)
dfyn_pool1

#pool 1001-2000
query2 = gql('''
query {
   pairs(first: 1000, skip: 1000, orderBy: reserve1, orderDirection: desc)
    {
      id
      reserve1
      token0 {
        id
        symbol
        decimals
      }
      token1 {
        id
        symbol
        decimals
      }
   }
}
''')

response2 = client.execute(query2)
dfyn_pool2 = pd.json_normalize(response2['pairs'],max_level=1)
dfyn_pool2

dfyn_pools = pd.concat([dfyn_pool1,dfyn_pool2], ignore_index=True)
dfyn_pools.columns = ['pool_address','tvl',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["dfyn" for x in range(len(dfyn_pools.index))])
dfyn_pools.insert(loc=0, column='Name', value=Name)
dfyn_pools.insert(loc=3, column='fee', value=0.3)
dfyn_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,dfyn,0xb3cd0bc759c3c3c76f9284852399b7862c95974b,1520525670046534.770838707,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0x98d0274895a6187e095859d80b6a1248a7fefb98,XMN,9
1,dfyn,0x9ea05b92a6af4878e8b2c4017d82039ce4ae126e,1471785859019268.042579210045238626,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0xddcb9ec8a14a55b1d8d312e6e9d6549cb32b77d9,ULTRALITE,18
2,dfyn,0x3bc29a33319f4b580644fd402728d48817120d52,1075280528038156.766292869873223578,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0xe8e96dc55573f308074569521e37f3bd532d2e9d,HYPERMATIC,18
3,dfyn,0xd78617714fb9606122ed6fa943686eda0c0ff09c,658311396887099.713929383,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0xb1cb4ac6b1685dcbe691b9246406748805aa6fa8,POLYSAFE,9
4,dfyn,0xe3769a998745d1e442381adbe4ecfcc96b2e5379,519630084447999.049854715719275708,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0xc3a32d7a1eab69c146742ecb309b4fcd422f58b3,METROQUICK,18
...,...,...,...,...,...,...,...,...,...,...
1709,dfyn,0x11d62b054e8c1214c63c80581b5375511711c2c7,0,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0x5034077b88879177989558c22abf6f4253b39177,HAUFO T,18
1710,dfyn,0x116e2751264aa966442d71943b5e10c6339171c7,0,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0x75d4688194ca968f23d31e1453a1c61eead99e97,ARTHX,18
1711,dfyn,0x10b0ff4a7f6485a77559a4d73be382f706306eb9,0,0.3,0x4c28f48448720e9000907bc2611f73022fdce1fa,WETH,18,0x9ffcf6a70c01d1e5dd6db55f1b63d84b3a863cfb,HAS,18
1712,dfyn,0x0eebe2d57f27ea12bd6ae5ea6a314b0e17c6bc31,0,0.3,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xa29c1718865b5bcd3278ae1da52c4aa9c0f28e4f,LEMON,18


In [ ]:
dfyn_pools.to_csv('dfyn_polygon.csv')

### Cafeswap

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/cafeswap-polygon',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, orderBy: syncAtTimestamp) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)
cafeswap_pool = pd.json_normalize(response1['pairs'],max_level=1)

cafeswap_pool.columns = ['pool_address',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["cafeswap" for x in range(len(cafeswap_pool.index))])
cafeswap_pool.insert(loc=0, column='Name', value=Name)
cafeswap_pool.insert(loc=2, column='fee', value=0.3)
cafeswap_pool

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,cafeswap,0xe8da714c1cbb787ac3f0168af205ed9a19ab6b94,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x723e866989cacc8efc346e1dcf5a9d73572bcfc8,Cafe-LP,18
1,cafeswap,0x8b2535b367972dc97c738d9e2f6428ae15ad2813,0.3,0x20d3922b4a1a8560e1ac99fba4fade0c849e2142,maWETH,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6
2,cafeswap,0x12f0fda9325717d3974fdfa4637522c971c14770,0.3,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,WBTC,8,0x692597b009d13c4049a947cab2239b7d6517875f,UST,18
3,cafeswap,0xf8749ebea669d877f430d85e6e75ab769f7f3e46,0.3,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xd6df932a45c0f255f85145f286ea0b292b21c90b,AAVE,18
4,cafeswap,0xe490cb385a63d9980bdc01dcd84a38081cd6f8d3,0.3,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,DAI,18,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,USDT,6
...,...,...,...,...,...,...,...,...,...
134,cafeswap,0x80f93221e875d14aeddfdfc778ca755ab3562ce3,0.3,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18,0xd6df932a45c0f255f85145f286ea0b292b21c90b,AAVE,18
135,cafeswap,0x64ac1107923413b6faf0168e2063c1d4cb01bc9b,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18
136,cafeswap,0xbb0dbc0e2b6fbef902bc046ecfc2d664658baafc,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6
137,cafeswap,0x4f03233c742ea20e73b46d5e33242b7770274ab8,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0xb5106a3277718ecad2f20ab6b86ce0fee7a21f09,pBREW,18


In [ ]:
cafeswap_pool.to_csv('cafeswap_polygon.csv')

### Dinoswap

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/dinoswap-polygon',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: desc) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)
dinoswap_pool = pd.json_normalize(response1['pairs'],max_level=1)

dinoswap_pool.columns = ['pool_address',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["dinoswap" for x in range(len(dinoswap_pool.index))])
dinoswap_pool.insert(loc=0, column='Name', value=Name)
dinoswap_pool.insert(loc=2, column='fee', value=0.3)
dinoswap_pool

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,dinoswap,0x4aecb53c491a4461e528de2db2e40924b4a04461,0.3,0xb52ecd25ecec0853fb4ffd112531af80370f45c6,QFFEED,9,0xd1f5dc19277d372f855eeaf5f0da061e6b0ea399,C.MILK,9
1,dinoswap,0xffe859f90217387863f77745d3517a17f0f9dc42,0.3,0x7433028d5f053ccc5ca917c269c3cf84c4f7e616,XUSD,18,0xd1f5dc19277d372f855eeaf5f0da061e6b0ea399,C.MILK,9
2,dinoswap,0x2056188575f3bb5baa688670e6475acfa6e09475,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x721973f03c3c720e724fdbfdeca3721a750ba764,MGT,18
3,dinoswap,0xcd5c2ec48f90ec142815927652a0595ece0a205c,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x203ebef6ca0598458dc79864ec2f546f49dcb229,314,14
4,dinoswap,0xb8b070d7f6ca43e361b52aa402ce8764496e7e34,0.3,0x721973f03c3c720e724fdbfdeca3721a750ba764,MGT,18,0xaa9654becca45b5bdfa5ac646c939c62b527d394,DINO,18
...,...,...,...,...,...,...,...,...,...
67,dinoswap,0xce621ce85eabc6dd95088b81cab683dfb4628864,0.3,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18,0xaa9654becca45b5bdfa5ac646c939c62b527d394,DINO,18
68,dinoswap,0xa8fb745919d959fbbdfab71b88e82bb4e6ba502e,0.3,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18
69,dinoswap,0x8a42efda52787fb80af6aea4d9d3d67d1c3d136f,0.3,0x5f29e75e240fa9b1d90679ea70ecafa86f089ef9,TEST2,18,0xe3b3de31aa8db814abcd2ba059c687dfab66a839,TEST1,18
70,dinoswap,0xc961ac0404f95141573dae224525ae936bd3bf64,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6


In [ ]:
dinoswap_pool.to_csv('dinoswap_polygon.csv')

### Fraxswap

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/fraxswap-polygon',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)
fraxswap_pool = pd.json_normalize(response1['pairs'],max_level=1)

fraxswap_pool.columns = ['pool_address',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["fraxswap" for x in range(len(fraxswap_pool.index))])
fraxswap_pool.insert(loc=0, column='Name', value=Name)
fraxswap_pool

,Name,pool_address,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,fraxswap,0xd2105fe5f1b631daf2398e918549758cd181ca7c,0x1a3acf6d19267e2d3e7f898f42803e90c9219062,FXS,18,0x45c32fa6df82ead1e2ef74d17b76547eddfaff89,FRAX,18
1,fraxswap,0xa964b92c690f0b8efad5b46a278a84bd0ef8ac01,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x45c32fa6df82ead1e2ef74d17b76547eddfaff89,FRAX,18
2,fraxswap,0x822f7585206464ea89aa37afaf6d8432ed6db220,0x45c32fa6df82ead1e2ef74d17b76547eddfaff89,FRAX,18,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18
3,fraxswap,0x214560ef46560db43c90214d4f0b6b3cc968e7b2,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x1a3acf6d19267e2d3e7f898f42803e90c9219062,FXS,18
4,fraxswap,0xfd515305d284ce7e62158d83f406927d53692238,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0x45c32fa6df82ead1e2ef74d17b76547eddfaff89,FRAX,18
5,fraxswap,0x39dd8f765a7b8e1b143fa795cd2cc52cc2749dfe,0x45c32fa6df82ead1e2ef74d17b76547eddfaff89,FRAX,18,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,DAI,18
6,fraxswap,0xe7837762b624a41a8b2db1ebe7b7dfabbc50845e,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x8c778917e4e02f6122ccd1198b3954d8c7bb2c58,PPeth,9
7,fraxswap,0x47525f4fc8a312b1e87ee5e19c8cf51f796c677e,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0xa57431b60940e0fc19cad78edb54d0e5501d8033,PTB,9
8,fraxswap,0x6cab0e54b35c4b7bad21261c6a979efedc93a90b,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0xf5707a2bca11e0572c9631faf1854289dc15f77c,SPME,18


In [ ]:
fraxswap_pool.to_csv('fraxswap_polygon.csv')

### Gravity

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/shayha271097/polygon-gravity-finance',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: desc) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)
gravity_pool = pd.json_normalize(response1['pairs'],max_level=1)

gravity_pool.columns = ['pool_address',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["gravity" for x in range(len(gravity_pool.index))])
gravity_pool.insert(loc=0, column='Name', value=Name)
gravity_pool

,Name,pool_address,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,gravity,0xf4eac0b904602de022e934bbb5c37131cb6cf6fb,0x874e178a2f3f3f9d34db862453cd756e7eab0381,GFI,18,0xa3fa99a148fa48d14ed51d610c367c61876997f1,miMATIC,18
1,gravity,0x72566b9e8ba90c02274a712e58c6810327170138,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x1b815d120b3ef02039ee11dc2d33de7aa4a8c603,WOO,18
2,gravity,0xd6265a06fa0ddcd1ac266740d44b897c7919dbda,0x874e178a2f3f3f9d34db862453cd756e7eab0381,GFI,18,0xf08c4ebae711242ca26c6c29116badbc9af123ee,GLP,18
3,gravity,0x3925ed8768c4528ecab0851761e49a70a7c47c9f,0x16eafd09eec2f2dfbc437fa5a55e82c27f370bed,JPYS,18,0x874e178a2f3f3f9d34db862453cd756e7eab0381,GFI,18
4,gravity,0x241ed15a37c85e531e83255ae719b8d2ea079ddb,0x874e178a2f3f3f9d34db862453cd756e7eab0381,GFI,18,0xb5c064f955d8e7f38fe0460c556a72987494ee17,QUICK,18
...,...,...,...,...,...,...,...,...
83,gravity,0xcdd1ad14b4c156aff8d833de1793f67d75237909,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,WBTC,8,0xdbf31df14b66535af65aac99c32e9ea844e14501,renBTC,8
84,gravity,0xe507a93b99120d916229ee6d4adb4204bbb04edc,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0x692597b009d13c4049a947cab2239b7d6517875f,UST,18
85,gravity,0x42286296c3ede3f6a0ec4e687939b017408cf321,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,USDT,6
86,gravity,0x4bb452b8b74864f44913f100bee280daa61ac298,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,DAI,18


In [ ]:
gravity_pool.to_csv('gravity_polygon.csv')

### Honeyswap

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/honeyswap-polygon',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

query1 = gql('''
query {
   liquidityPools(first: 1000, orderBy: totalValueLockedUSD, orderDirection: desc) {
    id
    totalValueLockedUSD
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

honeyswap_pool = pd.concat([df11[['id', 'totalValueLockedUSD']],df14,df12,df13],axis=1)

honeyswap_pool.columns = ['pool_address','tvl','fee',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["honeyswap" for x in range(len(honeyswap_pool.index))])
honeyswap_pool.insert(loc=0, column='Name', value=Name)
honeyswap_pool

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,honeyswap,0x0c787944946d22922c9f41c477cc539700d35bb2,29394.4398785531070646928147607092,0.3,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18,0xd6df932a45c0f255f85145f286ea0b292b21c90b,AAVE,18
1,honeyswap,0x9283508891958dc72425339b1c25517cbc37b347,25149.03974,0.3,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xe83ce6bfb580583bd6a62b4be7b34fc25f02910d,MABBC,8
2,honeyswap,0xd1782db702ab9d977c96c8bc9b038aa3aa937119,7136.359138,0.3,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xe7879bafa6a41f919f145a97207e54839d34c9ba,MSTRM,18
3,honeyswap,0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34,4770.524973553418305406080566412266,0.3,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,WBTC,8,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18
4,honeyswap,0xd862db749534d539713b2c392421fe5a8e43e19e,4479.222482,0.3,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18
...,...,...,...,...,...,...,...,...,...,...
296,honeyswap,0x0ac18328a7bd416b02b5658c05add6e30cac4820,0,0.3,0x6f57ec3c983a04c9bf9958c9e908c39a3cd0870d,AMIS,9,0x7dff46370e9ea5f0bad3c4e29711ad50062ea7a4,SOL,18
297,honeyswap,0x09c411fa74e706be846a9dd9628e683c6d742330,0,0.3,0x0c787944946d22922c9f41c477cc539700d35bb2,Wrapped Ether/Aave (PoS),18,0xb371248dd0f9e4061ccf8850e9223ca48aa7ca4b,HNY,18
298,honeyswap,0x08a1f06685a90295115f650e87a41e2a8b9c130e,0,0.3,0x64c1b62fe216b167bc7fd5b217630d494954b4c4,ELONINDEX,18,0xe0339c80ffde91f3e20494df88d4206d86024cdf,ELON,18
299,honeyswap,0x05cde5c5d656b46832e0f8f5863b2cf3081c0041,0,0.3,0xb371248dd0f9e4061ccf8850e9223ca48aa7ca4b,HNY,18,0xd862db749534d539713b2c392421fe5a8e43e19e,USD Coin (PoS)/Wrapped Ether,18


In [ ]:
honeyswap_pool.to_csv('honeyswap_polygon.csv')

### Jetswap

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/jetswap-polygon',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: desc) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)
jetswap_pool = pd.json_normalize(response1['pairs'],max_level=1)

jetswap_pool.columns = ['pool_address',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["jetswap" for x in range(len(jetswap_pool.index))])
jetswap_pool.insert(loc=0, column='Name', value=Name)
jetswap_pool.insert(loc=2, column='fee', value=0.3)
jetswap_pool

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,jetswap,0x71dfe7a36cc877da76cc66e9abc2b4b378ced9b1,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x8c31012d3f28e6e210efb5a6caf0d4cfe0c95fa6,HLM,9
1,jetswap,0x2da279b5ecd59ccfb1df3fda071932ea4e97df24,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0xce2b6e121fc2c9a5b3a939d017f7ea7b982ed32f,SYT,18
2,jetswap,0x7f4e96eddb673040c240edb57f6128363acbc04c,0.3,0x593c4976ef151ea8824a5107f1c43b792b82a060,CPN,18,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,USDT,6
3,jetswap,0x16eb4e9600c29b8c22fbe2fa40ac027daac970ba,0.3,0x0b265919f1b9285fe283010a874b2beae32d731e,CGS,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18
4,jetswap,0xcd0a8a7780f833aaab1480f3ef37caa19d30d33c,0.3,0x07cc1cc3628cc1615120df781ef9fc8ec2feae09,BEPRO,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18
...,...,...,...,...,...,...,...,...,...
379,jetswap,0x951e38875a93df95bbd24fe31f409b7933b35bed,0.3,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18
380,jetswap,0xc7f1b47f4ed069e9b34e6bd59792b8abf5a66339,0.3,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,USDT,6
381,jetswap,0xe4eb4261a4852c09055a6a94e2f7e8289425f0ef,0.3,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,DAI,18
382,jetswap,0xfeff91c350bb564ca5dc7d6f7dcd12ac092f94ff,0.3,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,WETH,18


In [ ]:
jetswap_pool.to_csv('jetswap_polygon.csv')

### Uniswap V3

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-polygon',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            inputTokens {
              id
    }
  }
      }
''')

response1 = client.execute(query1)
df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df1 = pd.concat([df11[['id', 'totalValueLockedUSD']],df12,df13],axis=1)
uniswap_pool1=df1

#pool 1001-2000
query2 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 1000){
            id
            totalValueLockedUSD
            inputTokens {
              id
    }
  }
      }
''')

response2 = client.execute(query2)
df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])
df2 = pd.concat([df21[['id', 'totalValueLockedUSD']],df22,df23],axis=1)
uniswap_pool2=df2

#pool 2001-3000
query3 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 2000){
            id
            totalValueLockedUSD
            inputTokens {
              id
    }
  }
      }
''')

response3 = client.execute(query3)
df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[1])
df3 = pd.concat([df31[['id', 'totalValueLockedUSD']],df32,df33],axis=1)
uniswap_pool3=df3

#pool 3001-4000
query4 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 3000){
            id
            totalValueLockedUSD
            inputTokens {
              id
    }
  }
      }
''')

response4 = client.execute(query4)
df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])
df4 = pd.concat([df41[['id', 'totalValueLockedUSD']],df42,df43],axis=1)
uniswap_pool4=df4

#pool 4001-5000
query5 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 4000){
            id
            totalValueLockedUSD
            inputTokens {
              id
    }
  }
      }
''')

response5 = client.execute(query5)
df51 = pd.json_normalize(response5['liquidityPools'],max_level=1)
df52 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[0])
df53 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[1])
df5 = pd.concat([df51[['id', 'totalValueLockedUSD']],df52,df53],axis=1)
uniswap_pool5=df5

#pool 5001-6000
query6 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 5000){
            id
            totalValueLockedUSD
            inputTokens {
              id
    }
  }
      }
''')

response6 = client.execute(query6)
df61 = pd.json_normalize(response6['liquidityPools'],max_level=1)
df62 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[0])
df63 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[1])
df6 = pd.concat([df61[['id', 'totalValueLockedUSD']],df62,df63],axis=1)
uniswap_pool6=df6

#Combination
uniswap_pools = pd.concat([uniswap_pool1,uniswap_pool2,uniswap_pool3,uniswap_pool4,uniswap_pool5,uniswap_pool6], ignore_index=True)
uniswap_pools['totalValueLockedUSD'] = uniswap_pools['totalValueLockedUSD'].apply(float)
uniswap_pools = uniswap_pools[uniswap_pools['totalValueLockedUSD'] > 0]
uniswap_pools.columns = ['pool_address', 'tvl', 'token1','token2']
uniswap_pools = uniswap_pools.reset_index(drop=True)
uniswap_pools

,pool_address,tvl,token1,token2
0,0x45dda9cb7c25131df268515131f647d726f50608,1.035741e+07,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
1,0x9ab9f658104467604b5afa9a3e1df62f35f7b208,7.820378e+06,0x12a4cebf81f8671faf1ab0acea4e3429e42869e7,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
2,0x50eaedb835021e4a108b7290636d62e9765cc6d7,7.449722e+06,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
3,0x0a63d3910ffc1529190e80e10855c4216407cc45,6.545997e+06,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xb0b195aefa3650a6908f15cdac7d92f8a5791b0b
4,0xa374094527e1673a86de625aa59517c5de346d32,5.170702e+06,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x2791bca1f2de4661ed88a30c99a7a9449aa84174
...,...,...,...,...
5995,0xc5d611ee1356535f0ee592eb14a8790348604749,3.590801e-15,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0xf8f9efc0db77d8881500bb06ff5d6abc3070e695
5996,0xc32a1fade383f7000a795dfddadc1afee6e081f7,3.566372e-15,0x03b54a6e9a984069379fae1a4fc4dbae93b3bccd,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
5997,0x1891ae3da30fd8864f47eff2f63f616c1d70e650,3.563087e-15,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0xb8e092d91d09f0f1123a40fb52a36882635d387c
5998,0x0f5d4f2aee8f71283cce7875187da935516982f7,3.474513e-15,0x10d6057f7331bf86933780fd244c552a38657fe4,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619


In [ ]:
uniswap_pools.to_csv('uniswap_v3_polygon.csv')

## ETH

### Uniswap V2

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)
uniswap_pool1 = pd.json_normalize(response1['pairs'],max_level=1)

#1001-2000 pool
query2 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc, skip: 1000) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response2 = client.execute(query2)
uniswap_pool2 = pd.json_normalize(response2['pairs'],max_level=1)

#2001-3000 pool
query3 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc, skip: 2000) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response3 = client.execute(query3)
uniswap_pool3 = pd.json_normalize(response3['pairs'],max_level=1)

#3001-4000 pool
query4 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc, skip: 3000) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response41 = client.execute(query4)
uniswap_pool4 = pd.json_normalize(response1['pairs'],max_level=1)

#4001-5000 pool
query5 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc, skip: 4000) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response5 = client.execute(query5)
uniswap_pool5 = pd.json_normalize(response5['pairs'],max_level=1)

#5001-6000 pool
query6 = gql('''
query {
   pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc, skip: 5000) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response6 = client.execute(query6)
uniswap_pool6 = pd.json_normalize(response6['pairs'],max_level=1)

uniswap_pool = pd.concat([uniswap_pool1,uniswap_pool2,uniswap_pool3,uniswap_pool4,uniswap_pool5,uniswap_pool6], ignore_index=True)
uniswap_pool.columns = ['pool_address','token1','token1_symbol','token1_decimals','token2','token2_symbol','token2_decimals']
Name = pd.Series(["uniswapv2_new" for x in range(len(uniswap_pool.index))])
uniswap_pool.insert(loc=0, column='Name', value=Name)
uniswap_pool.insert(loc=2, column='fee', value=0.3)
uniswap_pool

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,uniswapv2_new,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,uniswapv2_new,0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee,0.3,0x8e870d67f660d95d5be530380d0ec0bd388289e1,PAX,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
2,uniswapv2_new,0x12ede161c702d1494612d19f05992f43aa6a26fb,0.3,0x06af07097c9eeb7fd685c692751d5c66db49c215,CHAI,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
3,uniswapv2_new,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0.3,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,uniswapv2_new,0x07f068ca326a469fc1d87d85d448990c8cba7df9,0.3,0x408e41876cccdc0f92210600ef50372656052a38,REN,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
...,...,...,...,...,...,...,...,...,...
5995,uniswapv2_new,0xec565af969ac889387a73844c2ab07ad17e2793a,0.3,0x4922a015c4407f87432b179bb209e125432e4a2a,XAUt,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
5996,uniswapv2_new,0x6557fc9c44b4a04b9ec86816f9f6fcd97b0e5a2f,0.3,0x11d8fabce53d02fe99e18630b9a6853a442369a2,FUEL,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5997,uniswapv2_new,0xb300a865a32308d2a02e095896b648320e5016cf,0.3,0xafe5f91790ff7cecfece925e15f444c5cfb13a9b,RFUEL,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5998,uniswapv2_new,0x041da6cf99ec1e2a88512707ad4fe64e0de286d6,0.3,0x37a756dd30219c8ec46b33128cb15f01fdcd37b4,YFP,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


In [ ]:
uniswap_pool.to_csv('uniswapv2_eth.csv')

### Uniswap V2 add

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v2-swap-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(
    first: 1000
    orderBy: createdTimestamp
    orderDirection: asc
  ) {
    id
    inputTokens {
      id
      symbol
      decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])

uniswapv2_pool1 = pd.concat([df11[['id']],df12,df13],axis=1)
uniswapv2_pool1

#pool 1001-2000
query2 = gql('''
query {
        liquidityPools(
    first: 1000
    orderBy: createdTimestamp
    orderDirection: asc
    skip: 1000
  ) {
    id
    inputTokens {
      id
      symbol
      decimals
    }
  }
      }
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])

uniswapv2_pool2 = pd.concat([df21[['id']],df22,df23],axis=1)
uniswapv2_pool2

#pool 2001-3000
query3 = gql('''
query {
        liquidityPools(
    first: 1000
    orderBy: createdTimestamp
    orderDirection: asc
    skip: 2000
  ) {
    id
    inputTokens {
      id
      symbol
      decimals
    }
  }
      }
''')

response3 = client.execute(query3)

df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[1])

uniswapv2_pool3 = pd.concat([df31[['id']],df32,df33],axis=1)
uniswapv2_pool3

#pool 3001-4000
query4 = gql('''
query {
        liquidityPools(
    first: 1000
    orderBy: createdTimestamp
    orderDirection: asc
    skip: 3000
  ) {
    id
    inputTokens {
      id
      symbol
      decimals
    }
  }
      }
''')

response4 = client.execute(query4)

df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])

uniswapv2_pool4 = pd.concat([df41[['id']],df42,df43],axis=1)
uniswapv2_pool4

#pool 4001-5000
query5 = gql('''
query {
        liquidityPools(
    first: 1000
    orderBy: createdTimestamp
    orderDirection: asc
    skip: 4000
  ) {
    id
    inputTokens {
      id
      symbol
      decimals
    }
  }
      }
''')

response5 = client.execute(query5)

df51 = pd.json_normalize(response5['liquidityPools'],max_level=1)
df52 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[0])
df53 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[1])

uniswapv2_pool5 = pd.concat([df51[['id']],df52,df53],axis=1)
uniswapv2_pool5

#pool 5001-6000
query6 = gql('''
query {
        liquidityPools(
    first: 1000
    orderBy: createdTimestamp
    orderDirection: asc
    skip: 5000
  ) {
    id
    inputTokens {
      id
      symbol
      decimals
    }
  }
      }
''')

response6 = client.execute(query6)

df61 = pd.json_normalize(response6['liquidityPools'],max_level=1)
df62 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[0])
df63 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[1])

uniswapv2_pool6 = pd.concat([df61[['id']],df62,df63],axis=1)
uniswapv2_pool6

#Combination
uniswapv2_pools = pd.concat([uniswapv2_pool1,uniswapv2_pool2,uniswapv2_pool3,uniswapv2_pool4,uniswapv2_pool5,uniswapv2_pool6], ignore_index=True)
uniswapv2_pools.columns = ['pool_address','token1','token1_symbol','token1_decimals','token2','token2_symbol','token2_decimals']
Name = pd.Series(["uniswapv2_add" for x in range(len(uniswapv2_pools.index))])
uniswapv2_pools.insert(loc=0, column='Name', value=Name)
uniswapv2_pools.insert(loc=2, column='fee', value=0.3)
uniswapv2_pools

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,uniswapv2_add,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,uniswapv2_add,0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee,0.3,0x8e870d67f660d95d5be530380d0ec0bd388289e1,PAX,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
2,uniswapv2_add,0x12ede161c702d1494612d19f05992f43aa6a26fb,0.3,0x06af07097c9eeb7fd685c692751d5c66db49c215,CHAI,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
3,uniswapv2_add,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0.3,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,uniswapv2_add,0x07f068ca326a469fc1d87d85d448990c8cba7df9,0.3,0x408e41876cccdc0f92210600ef50372656052a38,REN,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
...,...,...,...,...,...,...,...,...,...
5995,uniswapv2_add,0xec565af969ac889387a73844c2ab07ad17e2793a,0.3,0x4922a015c4407f87432b179bb209e125432e4a2a,XAUt,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
5996,uniswapv2_add,0x6557fc9c44b4a04b9ec86816f9f6fcd97b0e5a2f,0.3,0x11d8fabce53d02fe99e18630b9a6853a442369a2,FUEL,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5997,uniswapv2_add,0xb300a865a32308d2a02e095896b648320e5016cf,0.3,0xafe5f91790ff7cecfece925e15f444c5cfb13a9b,RFUEL,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5998,uniswapv2_add,0x041da6cf99ec1e2a88512707ad4fe64e0de286d6,0.3,0x37a756dd30219c8ec46b33128cb15f01fdcd37b4,YFP,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


In [ ]:
uniswapv2_pools.to_csv('uniswap_v2_add_eth.csv')

# BSC

### Pancakeswap V2

In [ ]:
transport = RequestsHTTPTransport(
    url='https://open-platform.nodereal.io/8c4aebac194d4b5790f49e1df0d83c2f/pancakeswap-free/graphql',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
   pairs(first: 1000, skip: 0, orderBy: trackedReserveBNB, orderDirection: desc)
    {
      id
      reserveUSD
      token0 {
        id
        symbol
        decimals
      }
      token1 {
        id
        symbol
        decimals
      }
   }
}
''')

response1 = client.execute(query1)
pancakeswapv2_pool1 = pd.json_normalize(response1['pairs'],max_level=1)
pancakeswapv2_pool1

#pool 1001-2000
query2 = gql('''
query {
   pairs(first: 1000, skip: 1000, orderBy: trackedReserveBNB, orderDirection: desc)
    {
      id
      reserveUSD
      token0 {
        id
        symbol
        decimals
      }
      token1 {
        id
        symbol
        decimals
      }
   }
}
''')

response2 = client.execute(query2)
pancakeswapv2_pool2 = pd.json_normalize(response2['pairs'],max_level=1)
pancakeswapv2_pool2

#pool 2001-3000
query3 = gql('''
query {
   pairs(first: 1000, skip: 2000, orderBy: trackedReserveBNB, orderDirection: desc)
    {
      id
      reserveUSD
      token0 {
        id
        symbol
        decimals
      }
      token1 {
        id
        symbol
        decimals
      }
   }
}
''')

response3 = client.execute(query3)
pancakeswapv2_pool3 = pd.json_normalize(response3['pairs'],max_level=1)
pancakeswapv2_pool3

# Combination
pancakeswap_pools = pd.concat([pancakeswapv2_pool1,pancakeswapv2_pool2,pancakeswapv2_pool3], ignore_index=True)
pancakeswap_pools.columns = ['pool_address','tvl',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["pancakeswap" for x in range(len(pancakeswap_pools.index))])
pancakeswap_pools.insert(loc=0, column='Name', value=Name)
pancakeswap_pools.insert(loc=3, column='fee', value=0.25)
pancakeswap_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,pancakeswap,0x52672c173Fa7F00d921437708927bFE9BB8892f5,54237751.24852849188830624253278815,0.25,0x04F46CDfE8DD348E41902eEF1aFF19AcE1661F4c,FTC,18,0x55d398326f99059fF775485246999027B3197955,USDT,18
1,pancakeswap,0x16b9a82891338f9bA80E2D6970FddA79D1eb0daE,37114055.16322286150773182527020067,0.25,0x55d398326f99059fF775485246999027B3197955,USDT,18,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,WBNB,18
2,pancakeswap,0x0eD7e52944161450477ee417DE9Cd3a859b14fD0,36287582.5761181526808308407463916,0.25,0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82,Cake,18,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,WBNB,18
3,pancakeswap,0xaDaD973f8920bc511d94aade2762284f621F1467,34532109.70899534500356195845983875,0.25,0x55d398326f99059fF775485246999027B3197955,USDT,18,0xeC1F55b5Be7Ee8c24Ee26B6Cc931ce4d7Fd5955C,RWT,18
4,pancakeswap,0x1CEa83EC5E48D9157fCAe27a19807BeF79195Ce1,31762826.20005767863360648566950396,0.25,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,WBNB,18,0xE283D0e3B8c102BAdF5E8166B73E02D96d92F688,ELEPHANT,9
...,...,...,...,...,...,...,...,...,...,...
2995,pancakeswap,0x578Abb4dFfa9e96e8Eb0D4f5Aa6919b1A261e14a,47218.49099238209639204883976060315,0.25,0xb9bFE9241aCFFBe7f5B06C4e668c4a43b8431D3a,AFC,18,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,WBNB,18
2996,pancakeswap,0xA2251651B4ec1a93e30f50e51C277288e84c351B,46410.05610909351831210861296118165,0.25,0x026dB614F070CB4C7E421dA22Df84eA1021236eB,ROW,18,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,WBNB,18
2997,pancakeswap,0x31FdF4333d2edCAdCebd25edd7DaBCB73B27e851,46396.43963980106975618036104888786,0.25,0x46D502Fac9aEA7c5bC7B13C8Ec9D02378C33D36F,WSPP,0,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,WBNB,18
2998,pancakeswap,0xbd9729b0476c496b9d40ffACE2cd31086770827D,46427.94868341930083008600786042759,0.25,0xb7E02a2384b6d3D111Aa950283d3563a66f7a006,BKM,9,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,WBNB,18


In [ ]:
pancake_3000 = pancakeswap_pools[['pool_address']]
pancake_3000.to_csv('pancake_3000.csv')

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/pancakeswap-v2-swap-bsc',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

query1 = gql('''
query {
   liquidityPools(first: 1000, skip: 0, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool1 = pd.concat([df11[['id']],df12,df13],axis=1)
pancakeswap_pool1

query2 = gql('''
query {
   liquidityPools(first: 1000, skip: 1000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response2 = client.execute(query2)
df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool2 = pd.concat([df21[['id']],df22,df23],axis=1)
pancakeswap_pool2

query3 = gql('''
query {
   liquidityPools(first: 1000, skip: 2000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response3 = client.execute(query3)
df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool3 = pd.concat([df31[['id']],df32,df33],axis=1)
pancakeswap_pool3

query4 = gql('''
query {
   liquidityPools(first: 1000, skip: 3000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response4 = client.execute(query4)
df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool4 = pd.concat([df41[['id']],df42,df43],axis=1)
pancakeswap_pool4

query5 = gql('''
query {
   liquidityPools(first: 1000, skip: 4000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response5 = client.execute(query5)
df51 = pd.json_normalize(response5['liquidityPools'],max_level=1)
df52 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[0])
df53 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool5 = pd.concat([df51[['id']],df52,df53],axis=1)
pancakeswap_pool5

query6 = gql('''
query {
   liquidityPools(first: 1000, skip: 5000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response6 = client.execute(query6)
df61 = pd.json_normalize(response6['liquidityPools'],max_level=1)
df62 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[0])
df63 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool6 = pd.concat([df61[['id']],df62,df63],axis=1)
pancakeswap_pool6

# Combination
pancakeswapv2_pools = pd.concat([pancakeswap_pool1,pancakeswap_pool2,pancakeswap_pool3,pancakeswap_pool4,pancakeswap_pool5,pancakeswap_pool6], ignore_index=True)
pancakeswapv2_pools.columns = ['pool_address',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["pancakeswap_v2" for x in range(len(pancakeswapv2_pools.index))])
pancakeswapv2_pools.insert(loc=0, column='Name', value=Name)
pancakeswapv2_pools.insert(loc=2, column='fee', value=0.25)
pancakeswapv2_pools

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,pancakeswap_v2,0x3132f224d52bd58b1b81acf62933d38641cf6f60,0.25,0xa040c6e3f7a3b9e51cb81bace48e0e2cd310685a,BIGsushi,0,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
1,pancakeswap_v2,0xe0f8914f8080deb039e0ff975b11413c5b2b9148,0.25,0x4481bb6c2a5de65268d29bb60a45f00d8e18fe12,XXCATENAXX,9,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
2,pancakeswap_v2,0xf5728f087239e5660c85b68dac0239a47c846814,0.25,0x485baa2e0f695ca23c2b0f481d9629ed3b49500b,ENS,9,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
3,pancakeswap_v2,0x98b0e9065fbf7e017fcdcc42a7c2efc5eddddc71,0.25,0x4a34be7fdc4ec5ed3debfad65fbd1920baaf0b99,REDVIPER,8,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
4,pancakeswap_v2,0x5b1e728ac03b69ce503cbd4d341fd417d1ef95de,0.25,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18,0xe651c13ff6e995305540dd29db02c5d9ca87e10b,MLFE,10
...,...,...,...,...,...,...,...,...,...
5995,pancakeswap_v2,0xd109e9b67f1f322f9c24bbb8e065d00fc15bad40,0.25,0x14c358b573a4ce45364a3dbd84bbb4dae87af034,DND,18,0x3ad9594151886ce8538c1ff615efa2385a8c3a88,SAFEMARS,9
5996,pancakeswap_v2,0x9f14ce639e5485e66689c4b48ac21db2d3ced88d,0.25,0x250cee3bbda066dbf93ba61005def0a3eea732da,ADAT,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
5997,pancakeswap_v2,0x249a4a4c4136b5b8643199b7f5384bee71b7ef43,0.25,0xa0bc742036f9f271d1bc0d019fd6054dbb0b0192,FOOD,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
5998,pancakeswap_v2,0xa9a545c98fbb82cbd28382baebe0c884191fc0ef,0.25,0x95c06b5fca76ccf86329708a1aae4afce44ea980,ElonPizza,0,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18


In [ ]:
pancakev2 = pancakeswapv2_pools.copy()
pancakev2.to_csv('pancake_6000.csv')

### Pancakeswap v3

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/pancakeswap-v3-swap-bsc',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

query1 = gql('''
query {
   liquidityPools(first: 1000, skip: 0, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool1 = pd.concat([df11[['id']],df12,df13],axis=1)
pancakeswap_pool1

query2 = gql('''
query {
   liquidityPools(first: 1000, skip: 1000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response2 = client.execute(query2)
df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool2 = pd.concat([df21[['id']],df22,df23],axis=1)
pancakeswap_pool2

query3 = gql('''
query {
   liquidityPools(first: 1000, skip: 2000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response3 = client.execute(query3)
df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool3 = pd.concat([df31[['id']],df32,df33],axis=1)
pancakeswap_pool3

query4 = gql('''
query {
   liquidityPools(first: 1000, skip: 3000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response4 = client.execute(query4)
df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool4 = pd.concat([df41[['id']],df42,df43],axis=1)
pancakeswap_pool4

query5 = gql('''
query {
   liquidityPools(first: 1000, skip: 4000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response5 = client.execute(query5)
df51 = pd.json_normalize(response5['liquidityPools'],max_level=1)
df52 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[0])
df53 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool5 = pd.concat([df51[['id']],df52,df53],axis=1)
pancakeswap_pool5

query6 = gql('''
query {
   liquidityPools(first: 1000, skip: 5000, orderBy: createdTimestamp, orderDirection: desc)
    {
      id
      inputTokens {
      id
      symbol
      decimals
    }
   }
}
''')

response6 = client.execute(query6)
df61 = pd.json_normalize(response6['liquidityPools'],max_level=1)
df62 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[0])
df63 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[1])

pancakeswap_pool6 = pd.concat([df61[['id']],df62,df63],axis=1)
pancakeswap_pool6

# Combination
pancakeswapv3_pools = pd.concat([pancakeswap_pool1,pancakeswap_pool2,pancakeswap_pool3,pancakeswap_pool4,pancakeswap_pool5,pancakeswap_pool6], ignore_index=True)
pancakeswapv3_pools.columns = ['pool_address',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["pancakeswap_v3" for x in range(len(pancakeswapv3_pools.index))])
pancakeswapv3_pools.insert(loc=0, column='Name', value=Name)
pancakeswapv3_pools.insert(loc=2, column='fee', value=0.3)
pancakeswapv3_pools

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,pancakeswap_v3,0xd802f346ae28178bd1d173362b1d90e56989ff5b,0.3,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18,0xf615c872e4d32bff2d176e87b78528357b6314ce,FLK,7
1,pancakeswap_v3,0xb47b7daaa271b9df3d75ffeb3b72d766e4474beb,0.3,0x903a801e55f92d0a519e0dad9c2527514db7af7e,x c,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
2,pancakeswap_v3,0x1e301d329706c705b6a28f4e934bf20d8ce9c47e,0.3,0x6df6520a335583f598623a8d438b8db858bb4752,LGC,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
3,pancakeswap_v3,0xc07b58e55ff1edbe009b16323b3a874c3284ea39,0.3,0x903a801e55f92d0a519e0dad9c2527514db7af7e,x c,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
4,pancakeswap_v3,0xf38acdbda366e6d4c7cb32923eb3ef91b99802fb,0.3,0x6772ec4daea9af17a56a51678ed784d2e586728a,BEE,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
...,...,...,...,...,...,...,...,...,...
5995,pancakeswap_v3,0xd414b47bac01f21405720b888a403f23b3f4f6be,0.3,0x15b5fb84beadefadf8a35425aba041130a8eeb0f,FHB,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
5996,pancakeswap_v3,0x42cbdca3c1daf21c04872dba3deb2a595d70f7e7,0.3,0x87fbacaad6c24a33d5c4ee5d76900fc5c9f0b548,FHB,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
5997,pancakeswap_v3,0x99975952bbdbe5eb84197aa9922d80dafe9b9ff9,0.3,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18,0xde59cf76f755beac73d1e415f44a8e07fe528d30,FAFO,4
5998,pancakeswap_v3,0x73ac41bbf9409708aaa04f903b1c2c00f9c10b56,0.3,0x593de0d2d2d0def561ffc0a9caa217daee6f3b49,TOBS,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18


In [ ]:
pancakeswapv3_pools.to_csv('pancakev3_6000.csv')

### New Pancakeswapv3

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/pancakeswap-v3-bsc',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

query1 = gql('''
query {
   liquidityPools(
    first: 1000
    orderBy: protocol__totalValueLockedUSD
    orderDirection: desc
  ) {
    id
    protocol {
      totalValueLockedUSD
    }
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

pancakeswap_pool1 = pd.concat([df11[['id']],df11['protocol.totalValueLockedUSD'],df14,df12,df13],axis=1)
pancakeswap_pool1

,id,protocol.totalValueLockedUSD,feePercentage,id,symbol,decimals,id,symbol,decimals
0,0xfffa1ad0ab7a891dfaf58728acfa9248b79f7c17,267301306.0996917266213572622441669,0.25,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18,0xc2f914093a6287aa2f225d255feafdb95ba7e99b,GORILA,18
1,0xffe930c707fa5b9ea4e74f3aa8ed4d851f8a297b,267301306.0996917266213572622441669,0.25,0x7130d2a12b9bcbfae4f2634d864a1ee1ce3ead9c,BTCB,18,0xd9907fcda91ac644f70477b8fc1607ad15b2d7a8,multiBTC,8
2,0xffe60b1ca197c385d288d0cf62809a41158a17b8,267301306.0996917266213572622441669,0.01,0x63c14da8bedd5ebc0b333ff67de3e955bcc458fc,BOB,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
3,0xffe581d0886797088af66881ade8f48fa0a7b253,267301306.0996917266213572622441669,1,0x3c928a6f6c1e9f547d5d1122a8d5591d8dd30d7b,PEPE,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
4,0xffd8885dc6cf05a1dc5783f2ea826a39f50f0f2a,267301306.0996917266213572622441669,1,0x08cd582b009ffd50582c644c40214e375a0e34e1,AP,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
...,...,...,...,...,...,...,...,...,...
995,0xe4eebfec1594b5ccaf872b8eb0450aa20c67b3a9,267301306.0996917266213572622441669,0.01,0x0ef3a583c49ac4f6ff64b73e36938584f6ce65cf,peee,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
996,0xe4ee5bd8df738729cd3494f071c0f134fa3f20a0,267301306.0996917266213572622441669,0.25,0x770e1953d5d6bd54107edc539f0edc0070a6ba1b,SONIC,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
997,0xe4e95da51fb99be98cd363808475c1e08f78dbe9,267301306.0996917266213572622441669,0.25,0x0eb3a705fc54725037cc9e008bdede697f62f335,ATOM,18,0xa5522ae272b74b014bc20ef7ef2e2b87af0787d8,SMU,18
998,0xe4e8e72542bb05883ed3a4a7911aae9e92c16197,267301306.0996917266213572622441669,1,0x7c3de0b37e11756f0805cb604cb50d95ecb6265e,ERA,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18


In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/pancakeswap-v3-bsc',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

query1 = gql('''
query {
   liquidityPools(
    first: 1000
    orderBy: totalValueLockedUSD
    orderDirection: desc
  ) {
    id
    totalValueLockedUSD
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

pancakeswap_pool1 = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
pancakeswap_pool1

query2 = gql('''
query {
   liquidityPools(
    first: 1000
    orderBy: totalValueLockedUSD
    orderDirection: desc
    skip: 1000
  ) {
    id
    totalValueLockedUSD
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df24 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

pancakeswap_pool2 = pd.concat([df21[['id', 'totalValueLockedUSD']],df24,df22,df23],axis=1)
pancakeswap_pool2

query3 = gql('''
query {
   liquidityPools(
    first: 1000
    orderBy: totalValueLockedUSD
    orderDirection: desc
    skip: 2000
  ) {
    id
    totalValueLockedUSD
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response3 = client.execute(query3)

df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df34 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

pancakeswap_pool3 = pd.concat([df31[['id', 'totalValueLockedUSD']],df34,df32,df33],axis=1)
pancakeswap_pool3

query4 = gql('''
query {
   liquidityPools(
    first: 1000
    orderBy: totalValueLockedUSD
    orderDirection: desc
    skip: 3000
  ) {
    id
    totalValueLockedUSD
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response4 = client.execute(query4)

df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])
df44 = pd.json_normalize(df41['fees'].apply(pd.Series)[2])

pancakeswap_pool4 = pd.concat([df41[['id', 'totalValueLockedUSD']],df44,df42,df43],axis=1)
pancakeswap_pool4

query5 = gql('''
query {
   liquidityPools(
    first: 1000
    orderBy: totalValueLockedUSD
    orderDirection: desc
    skip: 4000
  ) {
    id
    totalValueLockedUSD
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response5 = client.execute(query5)

df51 = pd.json_normalize(response5['liquidityPools'],max_level=1)
df52 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[0])
df53 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[1])
df54 = pd.json_normalize(df51['fees'].apply(pd.Series)[2])

pancakeswap_pool5 = pd.concat([df51[['id', 'totalValueLockedUSD']],df54,df52,df53],axis=1)
pancakeswap_pool5

query6 = gql('''
query {
   liquidityPools(
    first: 1000
    orderBy: totalValueLockedUSD
    orderDirection: desc
    skip: 5000
  ) {
    id
    totalValueLockedUSD
    fees {
      feePercentage
    }
    inputTokens {
      id
      symbol
      decimals
    }
  }
}
''')

response6 = client.execute(query6)

df61 = pd.json_normalize(response6['liquidityPools'],max_level=1)
df62 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[0])
df63 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[1])
df64 = pd.json_normalize(df61['fees'].apply(pd.Series)[2])

pancakeswap_pool6 = pd.concat([df61[['id', 'totalValueLockedUSD']],df64,df62,df63],axis=1)
pancakeswap_pool6

# Combination
pancakeswapv3_pools = pd.concat([pancakeswap_pool1,pancakeswap_pool2,pancakeswap_pool3,pancakeswap_pool4,pancakeswap_pool5,pancakeswap_pool6], ignore_index=True)
pancakeswapv3_pools.columns = ['pool_address', 'tvl', 'fee',
                               'token1','token1_symbol','token1_decimals',
                               'token2','token2_symbol','token2_decimals']
Name = pd.Series(["pancakeswap_v3" for x in range(len(pancakeswapv3_pools.index))])
pancakeswapv3_pools.insert(loc=0, column='Name', value=Name)
pancakeswapv3_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,pancakeswap_v3,0x4f31fa980a675570939b737ebdde0471a4be40eb,38050389.314391040254740653,0.05,0x55d398326f99059ff775485246999027b3197955,USDT,18,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,USDC,18
1,pancakeswap_v3,0x36696169c63e42cd08ce11f5deebbcebae652050,35202204.48967903040825598279692012,0.05,0x55d398326f99059ff775485246999027b3197955,USDT,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
2,pancakeswap_v3,0x4f3126d5de26413abdcf6948943fb9d0847d9818,33103098.31064416242730495,0.01,0x55d398326f99059ff775485246999027b3197955,USDT,18,0xe9e7cea3dedca5984780bafc599bd69add087d56,BUSD,18
3,pancakeswap_v3,0xd4dca84e1808da3354924cd243c66828cf775470,25056930.00008562209427050167840185,0.25,0x2170ed0880ac9a755fd29b2688956bd959f933f8,ETH,18,0x7130d2a12b9bcbfae4f2634d864a1ee1ce3ead9c,BTCB,18
4,pancakeswap_v3,0x7d05c84581f0c41ad80ddf677a510360bae09a5a,11721759.63575537369898436347446085,0.25,0x2170ed0880ac9a755fd29b2688956bd959f933f8,ETH,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
...,...,...,...,...,...,...,...,...,...,...
5995,pancakeswap_v3,0x285021ac3a3947b69b0a5c8a146c76541245135c,0.0005539288997363145513029601022101453,0.05,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18,0xdf82b5879c21dbaa916e1ebc6049869b3206bcc0,TED,18
5996,pancakeswap_v3,0x006941ebd97f64548a5415f0574d5cd236f81ef8,0.0005538480401578616794815302378693945,0.05,0x04ad0be41b578acaca6c4e00d927999d464f492a,CEX,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
5997,pancakeswap_v3,0x1c66f0ec9576c34f57c93d00ce2cf98ade1e03b8,0.0005520403877456385699991495091769627,0.05,0xa883b15bae43ace7053f5c2baca565fe52129600,PEPE,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18
5998,pancakeswap_v3,0xe9f3b2a474ef0bb71dd30c1957003d060fd86d75,0.000551773008177008453003007995364183,0.05,0x593bd966c2e98561f4c3b1ef809f592c0dc203fb,ZK,18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,WBNB,18


In [ ]:
pancakeswapv3_pools.to_csv('new_pancakev3.csv')